<a href="https://colab.research.google.com/github/CARLOS-DANIEL-GIL-GUZMAN/Taller-1/blob/main/Parcial2_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok


up to date, audited 23 packages in 577ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached websoc

#Punto 2

In [135]:
%%writefile Punto2.py
#Código Dashboard
import streamlit as st
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import joblib
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# Título de la aplicación
st.title('Parcial 2 Señales y Sistemas')

# Descripción o texto en la app
st.write("Punto 2:")

# Ingreso de la amplitud A y la frecuencia Fo
A = float(st.text_input("Ingrese un valor de entero positivo para A:", value=1))
Fo = float(st.text_input("Ingrese un valor de frecuencia positiva para Fo:", value=1))

# Simulación de la señal de entrada
ti = -1 / (2 * Fo)  # tiempo inicial
tf = 1 / (2 * Fo)   # tiempo final
T = tf - ti         # periodo total (duración del intervalo)
wo = 2 * np.pi / T  # frecuencia angular
st.write('Frecuencia angular (wo):', wo)
st.write('Frecuencia (Fo):', Fo)

Fs = 30 * Fo  # frecuencia de muestreo (respetar Nyquist, Fs >= 2Fo)
tv = np.arange(ti, tf, 1 / Fs)  # vector de tiempo
Nm = len(tv)  # número de muestras en el tiempo

# Señal de entrada |A sin(2πF_ot)|^2
xo = abs(A * np.sin(2 * np.pi * Fo * tv))**2  # señal |A sin(2πF_ot)|^2

# Gráfico de la señal
st.write("Gráfica de la señal de entrada:")
st.line_chart(xo)

# Simulación del espectro y la reconstrucción
# Construcción de bases serie exponencial
N = 10  # número de armónicos
wo = 2 * np.pi / T  # frecuencia fundamental

# Definir bases
phin = np.zeros((Nm, 2 * N + 1), dtype=np.complex_)  # matriz para guardar bases
for n in range(-N, N + 1, 1):
    phin[:, n + N] = np.exp(1j * n * wo * tv)  # base de Fourier en el intervalo de interés

# Espectro de la señal - Serie exponencial
# Calcular el espectro para la señal |A sin(2πF_ot)|^2
Cn = np.zeros(2 * N + 1, dtype=np.complex_)
nv = np.linspace(-N, N, 2 * N + 1)  # vector de número de armónicos
Cn[N] = A**2 / 2  # nivel DC C_0
Cn[N - 2] = A**2 / 4  # armónico en -2
Cn[N + 2] = A**2 / 4  # armónico en +2

# Convertir la magnitud del espectro a decibelios
magnitud_dB = 20 * np.log10(abs(Cn) + 1e-12)  # +1e-12 para evitar logaritmo de cero

# Graficar espectro: Magnitud en dB, Fase, Parte Real y Parte Imaginaria
st.write("Espectro de la señal:")

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Magnitud en decibelios
axs[0, 0].stem(nv, magnitud_dB, 'r', use_line_collection=True)
axs[0, 0].set_xlabel(r'$nw_o$[rad/s]')
axs[0, 0].set_ylabel('Magnitud [dB]')
axs[0, 0].set_title('Magnitud del Espectro en Decibelios')
axs[0, 0].grid()

# Fase en radianes
axs[0, 1].stem(nv, np.angle(Cn), 'r', use_line_collection=True)
axs[0, 1].set_xlabel(r'$nw_o$[rad/s]')
axs[0, 1].set_ylabel('Fase [rad]')
axs[0, 1].set_title('Fase del Espectro')
axs[0, 1].grid()

# Parte Real
axs[1, 0].stem(nv, np.real(Cn), 'b', use_line_collection=True)
axs[1, 0].set_xlabel(r'$nw_o$[rad/s]')
axs[1, 0].set_ylabel('Real$\{C_n\}$')
axs[1, 0].set_title('Parte Real del Espectro')
axs[1, 0].grid()

# Parte Imaginaria
axs[1, 1].stem(nv, np.imag(Cn), 'g', use_line_collection=True)
axs[1, 1].set_xlabel(r'$nw_o$[rad/s]')
axs[1, 1].set_ylabel('Imaginario$\{C_n\}$')
axs[1, 1].set_title('Parte Imaginaria del Espectro')
axs[1, 1].grid()

st.pyplot(fig)

# Reconstrucción de la señal usando las bases de Fourier
x_reconstruida = np.zeros(Nm, dtype=np.complex_)  # Definir como complejo
for n in range(-N, N + 1):
    x_reconstruida += Cn[n + N] * np.exp(1j * n * wo * tv)
x_reconstruida = np.real(x_reconstruida)  # Solo la parte real

# Cálculo del error relativo
error_relativo = np.abs(xo - x_reconstruida) / np.abs(xo)

# Graficar error relativo
st.write("Error relativo entre la señal original y la reconstruida:")
st.line_chart(error_relativo)

# Mostrar el error relativo máximo y promedio
st.write(f"Error relativo máximo: {np.max(error_relativo):.4f}")
st.write(f"Error relativo promedio: {np.mean(error_relativo):.4f}")


Overwriting Punto2.py


In [136]:
token="2loMihbb5sGRhDJt5EHXjiJaVsd_3tHfRgkUbZitpjScYWL4G"

In [137]:
# from pyngrok import ngrok

# # Set authentication token (unique per user)
# ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
# !nohup streamlit run Punto2.py --server.port 5011 &

# # Start ngrok tunnel to expose the Streamlit server
# ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
# print(' * Tunnel URL:', ngrok_tunnel.public_url)

#Punto 3


In [138]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok


up to date, audited 23 packages in 905ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached websoc

In [139]:
%%writefile Punto3.py
import os
import yt_dlp as youtube_dl
import subprocess
import soundfile as sf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# Título de la aplicación
st.title('Parcial 2 Señales y Sistemas')

# Descripción o texto en la app
st.write("Punto 3:")

# Función para descargar mp3 desde YouTube
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    return filename

# Subir archivo Excel con enlaces a videos de YouTube
uploaded_file = st.file_uploader("Sube un archivo Excel con enlaces de YouTube", type="xlsx")

if uploaded_file:
    X = pd.read_excel(uploaded_file)
    st.write("Primeras filas del archivo:")
    st.dataframe(X.head())

    # Crear carpeta para resultados
    os.makedirs("results", exist_ok=True)

    # Descargar y procesar el primer video
    n = 0  # Usamos el primer video para el ejemplo
    name_ = f'results/{X.loc[n, "band"]}_{n}_{X.loc[n, "type_num"]}'
    st.write(f"Descargando video: {X.loc[n, 'band']}")

    mp3_file = download_ytvid_as_mp3(X.loc[n, 'link'], name_)
    st.write("Conversión a WAV...")
    subprocess.call(['ffmpeg', '-y', '-i', mp3_file, f'{name_}.wav'])

    # Cargar archivo WAV y mostrar algunos datos
    wav_file = f"{name_}.wav"
    x, fs = sf.read(wav_file)
    st.write(f"Frecuencia de muestreo: {fs} Hz")
    st.audio(wav_file)

    # Partición del archivo de audio
    ts = 5  # Duración del segmento en segundos
    t_segment = np.arange(0, ts, 1/fs)
    x_t = x[:int(ts*fs), :]  # Primer segmento de audio

    st.write("Reproduciendo segmento de audio:")
    st.audio(x_t.T, sample_rate=fs)

    # Generación y visualización de la señal de portadora y mensaje
    xm = x_t[:, 0]  # Canal 1
    Fc = 15000  # Frecuencia de portadora
    Ac = max(abs(xm))  # Amplitud de la portadora
    c = Ac * np.sin(2 * np.pi * Fc * t_segment)

    # Graficar portadora y mensaje
    fig, ax = plt.subplots()
    ax.plot(t_segment, c, label='Portadora')
    ax.plot(t_segment, xm, label='Mensaje')
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    ax.legend()
    st.pyplot(fig)

    # Modulación AM
    y = (1 + xm / Ac) * c
    st.write("Reproduciendo señal modulada en AM:")
    st.audio(y, sample_rate=fs)

    # Espectros de las señales
    Xfc = np.fft.fft(c)
    Xfm = np.fft.fft(xm)
    Xfy = np.fft.fft(y)
    vfre = np.fft.fftfreq(len(c), 1/fs)

    # Graficar espectros
    fig, ax = plt.subplots()
    ax.plot(vfre, np.abs(Xfy), label='Modulada')
    ax.plot(vfre, np.abs(Xfm), label='Mensaje')
    ax.plot(vfre, np.abs(Xfc), label='Portadora')
    ax.set_xlim([0, 30000])
    ax.set_xlabel('Frecuencia [Hz]')
    ax.set_ylabel('Magnitud')
    ax.legend()
    st.pyplot(fig)


Overwriting Punto3.py


In [140]:
#token="2loMihbb5sGRhDJt5EHXjiJaVsd_3tHfRgkUbZitpjScYWL4G"

In [141]:
# from pyngrok import ngrok

# # Set authentication token (unique per user)
# ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
# !nohup streamlit run Punto3.py --server.port 5012 &

# # Start ngrok tunnel to expose the Streamlit server
# ngrok_tunnel = ngrok.connect(addr='5012', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
# print(' * Tunnel URL:', ngrok_tunnel.public_url)

#Punto 4

In [201]:
%%writefile Punto4.py
import streamlit as st
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import io

# Título de la aplicación
st.title('Parcial 2 Señales y Sistemas')

# Descripción o texto en la app
st.write("Punto 4:")

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

# Parámetros del sistema
sampling_rate = 10000  # Frecuencia de muestreo
T = 1.0 / sampling_rate  # Intervalo de tiempo
t = np.linspace(0, 1.0, sampling_rate)

# Generar señal del rectificador de onda completa
def rectificador_onda_completa(t, f, R, C=None):
    # Onda senoidal fundamental
    signal = np.abs(np.sin(2 * np.pi * f * t))

    if C is not None:
        # Respuesta para carga RC
        tau = R * C
        signal = signal * np.exp(-t / tau)

    return signal

# Calcular la FFT y los armónicos
def calcular_fft(signal, sampling_rate):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    return xf, np.abs(yf[:N // 2])

# Calcular THD
def calcular_thd(yf):
    V1 = yf[1]  # El primer armónico
    harmonics = np.sqrt(np.sum(yf[2:]**2))
    thd = harmonics / V1
    return thd

# Interfaz gráfica de Streamlit
st.title("Simulación de Rectificador de Onda Completa y Cálculo del THD")

# Parámetros de la simulación
st.subheader("Parámetros de la simulación")
f = st.slider("Frecuencia de la señal de entrada (Hz):", min_value=10, max_value=100, value=60)  # Frecuencia de la señal
R = st.slider("Ingrese el valor de la Resistencia R (ohmios):", min_value=0.0, max_value=100.0, value=10.0, step=0.1)  # Valor de R
C = st.slider("Ingrese el valor de la Capacitancia C (faradios):", min_value=0.0, max_value=10.0, value=1.0, step=0.1)  # Valor de C

# Simulación para carga resistiva pura
signal_resistiva = rectificador_onda_completa(t, f, R)

# Simulación para carga RC en serie
signal_rc = rectificador_onda_completa(t, f, R, C)

# FFT de ambas señales
xf_res, yf_res = calcular_fft(signal_resistiva, sampling_rate)
xf_rc, yf_rc = calcular_fft(signal_rc, sampling_rate)

# Calcular THD
thd_resistiva = calcular_thd(yf_res)
thd_rc = calcular_thd(yf_rc)

# Mostrar resultados de THD
st.write(f"THD para carga resistiva pura: {thd_resistiva:.4f}")
st.write(f"THD para carga RC: {thd_rc:.4f}")

# Graficar señales y su espectro
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Señal en el dominio del tiempo
axs[0, 0].plot(t[:500], signal_resistiva[:500])
axs[0, 0].set_title("Señal con carga resistiva pura")
axs[0, 0].set_xlabel("Tiempo [s]")
axs[0, 0].set_ylabel("Amplitud")

axs[0, 1].plot(t[:500], signal_rc[:500])
axs[0, 1].set_title("Señal con carga RC")
axs[0, 1].set_xlabel("Tiempo [s]")
axs[0, 1].set_ylabel("Amplitud")

# FFT de las señales
axs[1, 0].plot(xf_res, yf_res)
axs[1, 0].set_title("FFT - Carga resistiva pura")
axs[1, 0].set_xlabel("Frecuencia [Hz]")
axs[1, 0].set_ylabel("Amplitud")

axs[1, 1].plot(xf_rc, yf_rc)
axs[1, 1].set_title("FFT - Carga RC")
axs[1, 1].set_xlabel("Frecuencia [Hz]")
axs[1, 1].set_ylabel("Amplitud")

plt.tight_layout()
st.pyplot(fig)


Overwriting Punto4.py


In [164]:
token="2loMihbb5sGRhDJt5EHXjiJaVsd_3tHfRgkUbZitpjScYWL4G"

In [194]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Punto4.py --server.port 5010 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5010', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://ed9a-34-125-231-142.ngrok-free.app


In [182]:
exit("Stopping the execution")